In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import missingno as msn
import datetime
from pandas_profiling import ProfileReport
import sklearn

In [2]:
df = pd.read_parquet("escooter_history_2022.parquet")
df.loc[df.temp >45, "temp"] = df.temp.div(100)

In [11]:
df

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer
0,2020-01-04 00:00:09,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,True
1,2020-01-04 00:00:41,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,True
2,2020-01-04 00:01:20,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,True
3,2020-01-04 00:04:12,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,True
4,2020-01-04 00:15:19,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,True
...,...,...,...,...,...,...,...,...,...
3755410,2022-01-03 23:58:10,0.0,1.0,"clear, few clouds",10.66,13.635,65.0,8.9981,True
3755411,2022-01-03 23:58:28,0.0,1.0,"clear, few clouds",10.66,13.635,65.0,8.9981,True
3755412,2022-01-03 23:58:46,0.0,1.0,"clear, few clouds",10.66,13.635,65.0,8.9981,True
3755413,2022-01-03 23:59:16,0.0,1.0,"clear, few clouds",10.66,13.635,65.0,8.9981,True


In [29]:
df_2 = df.groupby(["registered_customer","weather", pd.Grouper(key="datetime", freq= "H")]).agg(
               humidity= ("humidity", "mean"), 
               windspeed= ("windspeed", "mean"), 
               temp= ("temp", "mean"), 
               atemp= ("atemp", "mean"), 
               reg_customer= ("registered_customer", "count")
               
).reset_index()

In [30]:
df_2["quaratl"] = df_2.datetime.dt.quarter

In [31]:
df_2

,registered_customer,weather,datetime,humidity,windspeed,temp,atemp,reg_customer,quaratl
0,False,"clear, few clouds",2020-01-04 00:00:00,81.000000,0.000000,9.84,14.395,3,1
1,False,"clear, few clouds",2020-01-04 01:00:00,80.000000,0.000000,9.02,13.635,8,1
2,False,"clear, few clouds",2020-01-04 02:00:00,80.000000,0.000000,9.02,13.635,5,1
3,False,"clear, few clouds",2020-01-04 03:00:00,75.000000,0.000000,9.84,14.395,3,1
4,False,"clear, few clouds",2020-01-04 06:00:00,80.000000,0.000000,9.02,13.635,2,1
...,...,...,...,...,...,...,...,...,...
33895,True,light snow or rain or thunderstorm,2022-01-01 10:00:00,81.000000,8.998100,10.66,12.120,235,1
33896,True,light snow or rain or thunderstorm,2022-01-01 11:00:00,93.000000,6.003200,8.20,12.120,230,1
33897,True,light snow or rain or thunderstorm,2022-01-01 12:00:00,99.965174,0.029867,8.20,12.120,201,1
33898,True,light snow or rain or thunderstorm,2022-01-01 13:00:00,100.000000,0.000000,8.20,12.120,185,1


In [42]:
path=["registered_customer","quaratl","weather"]
fig = px.sunburst(df_2,path=path,values="reg_customer",color="weather")
fig.update_layout(title= dict(text= "<b>Sunburst on registered_customer, quarter, weather</b>", y=0.98, x=0.46))

In [34]:
fig